#Load required inputs

In [24]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.figure import Figure 
from matplotlib import dates as mdates
import datetime
from ipywidgets import interact, interactive, fixed, Layout, HBox, VBox
from IPython.display import clear_output, display
import ipywidgets as widgets
from matplotlib import rcParams
import matplotlib.ticker as mtick
import snowflake.connector
from IPython.core.interactiveshell import InteractiveShell
rcParams.update({'figure.autolayout': True})
pd.options.mode.chained_assignment = None


In [25]:
#Config

In [26]:
config = {
    "user": "SBARBER",
    "password": "1Likespurs",
    "account": "rimes-aws_eu_west_01",
    "warehouse": "PUBLIC_WH",
    "session_parameters":{
       
    }
}
   

sf_con = snowflake.connector.connect(**config)

def query_sf(sf_con, sql, params=None):
    cur = sf_con.cursor()
    try:
        if params:
            cur.execute(sql, params)  # Execute with parameters
        else:
            cur.execute(sql)  # Execute without parameters
        return cur.fetch_pandas_all()
    except Exception as e:
        print(f"Error executing query: {e}")
    finally:
        cur.close()



In [27]:
#Load data

In [28]:
#load etf and index constituent data


query_params = {
    'target_date': '2024-02-13'
}

indexcons = """
SELECT IndexID, Isin, Weight, Effective_Date, view_type 
FROM indexdepot_ftp_euwest1.public.rimes_idx_constituents
WHERE Effective_Date = %(target_date)s AND view_type = 'COB'  and Feed_Version='1'
"""



# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
idx_cons = query_sf(sf_con, indexcons,query_params)




etfcons = """
SELECT concat(etfsource,'|',etfsymbol) as etfsymbol, Isin, Iwght as Weight, Date as Effective_Date
FROM ETFDEPOT_FTP_EUWEST1.PUBLIC.PRICING_ETF
where Date=%(target_date)s and Version='0' and Isin is not null
"""



# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
etf_cons = query_sf(sf_con, etfcons, query_params)

etf_cons = etf_cons[etf_cons['WEIGHT'] > 0]



idxreturns = """
    SELECT distinct
        Date, IndexID, IndexReturn1D
    FROM indexdepot_ftp_euwest1.public.RIMES_IDX_LEVEL i where Feed_Version='1'

"""


# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
idx_returns = query_sf(sf_con, idxreturns, query_params)



etfreturns = """
    select * from (SELECT
    concat(n.etfsource,'|',n.etfsymbol) as etfsymbol,
    n.pricedate AS Date,
    n.NAV,
    LAG(n.NAV) OVER (PARTITION BY n.ETFSYMBOL ORDER BY n.pricedate ASC) AS PrevNAV,
    (n.NAV - LAG(n.NAV) OVER (PARTITION BY n.ETFSYMBOL ORDER BY n.pricedate ASC)) 
    / NULLIF(LAG(n.NAV) OVER (PARTITION BY n.ETFSYMBOL ORDER BY n.pricedate ASC), 0) AS ETFReturn1D
FROM ETFDEPOT_FTP_EUWEST1.PUBLIC.NAV_ETF n where Version='0') where ETFReturn1D is not null
"""


# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
etf_returns = query_sf(sf_con, etfreturns, query_params)




# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
idx_returns = query_sf(sf_con, idxreturns, query_params)



etfcatalog = """
    SELECT distinct
        concat(etfsource,'|',etfsymbol) as etfsymbol, desc as description, baseccy
    FROM ETFDEPOT_FTP_EUWEST1.PUBLIC.NAV_ETF e  where Version='0'
"""


# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
etf_catalog = query_sf(sf_con, etfcatalog, query_params)


idxcatalog = """
    SELECT distinct
        indexid, description, indexccy, returntype
    FROM indexdepot_ftp_euwest1.public.RIMES_IDX
    WHERE  Date=%(target_date)s
"""


# Use the query_sf function to execute the SQL and fetch the results into a DataFrame
idx_catalog = query_sf(sf_con, idxcatalog, query_params)


# Restriction so we can match on non variants
idx_catalog['MatchID'] = idx_catalog['INDEXID'].apply(lambda x: '|'.join(x.split('|')[:2]))


In [29]:
#Get index to index overlap

In [30]:
import pandas as pd

def analyze_index_overlap(idx_id, idx_cons, idx_catalog):
    """
    Analyze overlap for a specific index against all other indexes.

    Parameters:
    - idx_id: The ID of the index to analyze.
    - idx_cons: DataFrame containing index constituents information.
    - idx_catalog: DataFrame with index catalog information.

    Returns:
    - DataFrame containing top overlaps for the specified index.
    """

    # Define a function to handle empty groups within the groupby.apply()
    def sum_weights(x):
        return pd.Series({
            'WEIGHTA': x['WEIGHT_x'].sum(),
            'WEIGHTB': x['WEIGHT_y'].sum()
        })
    
    # Filter idx_cons for the current index
    filtered_idx_cons = idx_cons[idx_cons['INDEXID'] == idx_id]

    # Create a merged DataFrame from filtered_idx_cons on idx_cons with conditions
    merged = filtered_idx_cons.merge(idx_cons, on='ISIN', suffixes=('_x', '_y'))


    # Filter out rows where IndexID_x == IndexID_y
    merged_filtered = merged[merged['INDEXID_x'] != merged['INDEXID_y']].copy()


    # Sum weights for each IndexID pair
    common_constituents = merged_filtered.groupby(['INDEXID_x', 'INDEXID_y'], as_index=False).apply(sum_weights)


    # Calculate the minimum weight directly within the merged_filtered DataFrame
    merged_filtered['MIN_WEIGHT'] = merged_filtered[['WEIGHT_x', 'WEIGHT_y']].min(axis=1)
  
    # Calculate the overlap coefficient for each pair of indexes
    overlap_coefficient = merged_filtered.groupby(['INDEXID_x', 'INDEXID_y'])['MIN_WEIGHT'].sum().reset_index()
    overlap_coefficient.rename(columns={'MIN_WEIGHT': 'OVERLAP'}, inplace=True)
    
    # Modify UniqueIndexB to capture the first two segments if necessary
    overlap_coefficient['UniqueIndexB'] = overlap_coefficient['INDEXID_y'].apply(lambda x: '|'.join(x.split('|')[:2]))
    
    # Calculate the maximum overlap for each unique Index B group
    max_overlap = overlap_coefficient.groupby(['INDEXID_x', 'UniqueIndexB'])['OVERLAP'].max().reset_index()
    
    # Rank and select top overlaps
    max_overlap['RANK'] = max_overlap.groupby('INDEXID_x')['OVERLAP'].rank(method='dense', ascending=False)
    
    # Filter to get the top 10 overlaps for each INDEXID_x
    top_overlaps = max_overlap[max_overlap['RANK'] <= 10].sort_values(by=['INDEXID_x', 'RANK'])
    
    # Assign MatchID for merging with idx_catalog
    top_overlaps['MatchID'] = top_overlaps['UniqueIndexB']
    
    # Merging top_overlaps with idx_catalog based on MatchID
    idx_overlaps = pd.merge(top_overlaps, idx_catalog, on='MatchID', how='left', suffixes=('', '_catalog'))
    
    return idx_overlaps

# Example usage:
# idx_id = 'YourIndexIDHere'
# idx_overlaps_for_index = analyze_index_overlap(idx_id, idx_cons, idx_catalog)
# print(idx_overlaps_for_index)





In [31]:
#Get ETF to Index overlap

In [32]:
import pandas as pd

def analyze_etf_index_overlap(idx_id, idx_cons, etf_cons, idx_catalog):
    """
    Analyze overlap for a specific ETF against index constituents.
    
    Parameters:
    - etf_id: The ID of the ETF to analyze.
    - idx_cons: DataFrame containing index constituents information.
    - etf_cons: DataFrame containing ETF constituents information.
    - idx_catalog: DataFrame with index catalog information.
    
    Returns:
    - DataFrame with overlap analysis for the specified ETF against indexes.
    """
    # Filter etf_cons for the specific ETF
    filtered_etf_cons = etf_cons[etf_cons['ETFSYMBOL'] == idx_id]
    
    # Adjust idx_cons columns for merging
    idx_cons_adjusted = idx_cons.rename(columns={'INDEXID': 'ETFSymbol', 'ISIN': 'ISIN', 'WEIGHT': 'WEIGHT', 'EFFECTIVE_DATE': 'EFFECTIVE_DATE'})

    # Merge filtered_etf_cons and idx_cons_adjusted on ISIN
    merged = filtered_etf_cons.merge(idx_cons_adjusted, on='ISIN', suffixes=('_etf', '_idx'))
    # Calculate minimum weight for each ISIN pair
    merged['MIN_WEIGHT'] = merged[['WEIGHT_etf', 'WEIGHT_idx']].min(axis=1)

    # Calculate total overlap for each index
    etf_index_overlaps = merged.groupby('ETFSymbol')['MIN_WEIGHT'].sum().reset_index()
    etf_index_overlaps.rename(columns={'ETFSymbol': 'INDEXID', 'MIN_WEIGHT': 'OVERLAP'}, inplace=True)
    
    # Assign ETFSymbol
    etf_index_overlaps['ETFSymbol'] = idx_id

    # Sort by OVERLAP in descending order and calculate RANK
    etf_index_overlaps = etf_index_overlaps.sort_values(by='OVERLAP', ascending=False).reset_index(drop=True)
    etf_index_overlaps['RANK'] = etf_index_overlaps['OVERLAP'].rank(method='dense', ascending=False)
    
    # Reorder columns to align with the expected output format
    etf_index_overlaps = etf_index_overlaps[['ETFSymbol', 'INDEXID', 'OVERLAP', 'RANK']]

    return etf_index_overlaps



In [33]:
#Index to ETF overlap

In [34]:
import pandas as pd

def analyze_index_etf_overlap(idx_id, idx_cons, etf_cons, idx_catalog):
    """
    Analyze overlap for a specific index against ETF constituents.
    """
    filtered_idx_cons = idx_cons[idx_cons['INDEXID'] == idx_id]
    
    # Adjust etf_cons columns for merging
    etf_cons_adjusted = etf_cons.rename(columns={'ETFSYMBOL': 'INDEXID', 'Isin': 'ISIN', 'WEIGHT': 'WEIGHT', 'EFFECTIVE_DATE': 'EFFECTIVE_DATE'})

    # Merge filtered_idx_cons and etf_cons_adjusted
    merged = filtered_idx_cons.merge(etf_cons_adjusted, on='ISIN', suffixes=('_idx', '_etf'))
    
    # Calculate minimum weight for each ISIN pair
    merged['MIN_WEIGHT'] = merged[['WEIGHT_idx', 'WEIGHT_etf']].min(axis=1)

    # Calculate total overlap for each ETF
    etf_overlaps = merged.groupby('INDEXID_etf')['MIN_WEIGHT'].sum().reset_index()
    etf_overlaps.rename(columns={'INDEXID_etf': 'ETFSymbol', 'MIN_WEIGHT': 'OVERLAP'}, inplace=True)
    
    # Assign INDEXID_x
    etf_overlaps['INDEXID_x'] = idx_id

    # Sort by OVERLAP in descending order and calculate RANK
    etf_overlaps = etf_overlaps.sort_values(by='OVERLAP', ascending=False).reset_index(drop=True)
    etf_overlaps['RANK'] = etf_overlaps['OVERLAP'].rank(method='dense', ascending=False)
    
    # Reorder columns to match idx_overlaps format
    etf_overlaps = etf_overlaps[['INDEXID_x', 'ETFSymbol', 'OVERLAP', 'RANK']]

    return etf_overlaps



In [35]:
#Get index to index returns

In [36]:
def complete_analysis(idx_id, idx_cons, idx_catalog, idx_returns):
    # Perform index overlap analysis
    idx_overlaps = analyze_index_overlap(idx_id, idx_cons, idx_catalog)

    # Proceed with merging returns data
    idx_returns['DATE'] = pd.to_datetime(idx_returns['DATE'])
    idx_overlaps_with_returns = pd.merge(idx_overlaps, idx_returns, left_on='INDEXID', right_on='INDEXID', how='left', suffixes=('', '_return'))
    idx_overlaps_with_all_returns = pd.merge(idx_overlaps_with_returns, idx_returns, left_on=['INDEXID_x', 'DATE'], right_on=['INDEXID', 'DATE'], how='left', suffixes=('_return_b', '_return_a'))

    # Rename and clean up columns as previously described
    idx_overlaps_with_all_returns.rename(columns={
        'INDEXRETURN1D_return_b': 'MatchedIndexReturn',
        'INDEXRETURN1D_return_a': 'OriginalIndexReturn',
        'INDEXID_return_b': 'MatchedIndex',
        'INDEXID_x': 'OriginalIndex'
    }, inplace=True)
    idx_analysis = idx_overlaps_with_all_returns.drop(['RANK', 'UniqueIndexB', 'MatchID', 'INDEXID_return_a'], axis=1)
    idx_analysis['InstrumentType']='Index'
    return idx_analysis



In [37]:
#Get index to etf returns

In [38]:
import pandas as pd

def complete_etf_analysis(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns):
    """
    Perform analysis to compare index overlaps with ETFs and their respective returns.

    Parameters:
    - idx_id: The ID of the index to analyze.
    - idx_cons: DataFrame containing index constituents information.
    - etf_cons: DataFrame containing ETF constituents information.
    - idx_catalog: DataFrame with index catalog information.
    - etf_returns: DataFrame containing ETF returns information.
    - idx_returns: DataFrame containing index returns information.

    Returns:
    - DataFrame containing overlaps and return comparisons.
    """
    # Generate ETF overlaps
    etf_overlaps = analyze_index_etf_overlap(idx_id, idx_cons, etf_cons, idx_catalog)

    # Prepare etf_returns DataFrame
    etf_returns['DATE'] = pd.to_datetime(etf_returns['DATE'])

    # Merge ETF overlaps with ETF returns
    etf_overlaps_with_returns = pd.merge(etf_overlaps, etf_returns, left_on='ETFSymbol', right_on='ETFSYMBOL', how='left', suffixes=('', '_etf'))
    
    # Prepare idx_returns DataFrame
    idx_returns['DATE'] = pd.to_datetime(idx_returns['DATE'])

    # Correct the merge to use consistent column names
    # Assuming 'IndexID' should match 'INDEXID_x' from etf_overlaps and 'DATE' is correct
    etf_overlaps_with_all_returns = pd.merge(etf_overlaps_with_returns, idx_returns, left_on=['INDEXID_x', 'DATE'], right_on=['INDEXID', 'DATE'], how='left')


    
    # Select relevant columns to match your desired output format
    etf_analysis = etf_overlaps_with_all_returns[['DATE','INDEXID_x', 'ETFSymbol', 'OVERLAP', 'ETFRETURN1D', 'INDEXRETURN1D']]
    etf_analysis.loc[:, 'InstrumentType'] = 'ETF'
    return etf_analysis

#idx_id='STXX|ESTX|PI'
#combined_statistics = complete_etf_analysis(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)
#print(combined_statistics)

In [39]:
#Get etf to index returns

In [40]:
import pandas as pd

def complete_etf_to_index_returns_analysis(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns):
    """
    Perform analysis to compare ETF overlaps with indexes and their respective returns.
    
    Parameters:
    - idx_id: The ID of the ETF to analyze.
    - idx_cons: DataFrame containing index constituents information.
    - etf_cons: DataFrame containing ETF constituents information.
    - idx_catalog: DataFrame with index catalog information.
    - etf_returns: DataFrame containing ETF returns information.
    - idx_returns: DataFrame containing index returns information.
    
    Returns:
    - DataFrame containing overlaps and return comparisons for an ETF against indexes.
    """
    # Generate ETF overlaps against indexes
    etf_index_overlaps = analyze_etf_index_overlap(idx_id, idx_cons, etf_cons, idx_catalog)

    # Prepare ETF returns DataFrame
    etf_returns['DATE'] = pd.to_datetime(etf_returns['DATE'])
    # Filter ETF returns for the specific ETF
    specific_etf_returns = etf_returns[etf_returns['ETFSYMBOL'] == idx_id]

    # Prepare Index returns DataFrame
    idx_returns['DATE'] = pd.to_datetime(idx_returns['DATE'])

    # Merge ETF overlaps with Index returns
    etf_index_overlaps_with_returns = pd.merge(etf_index_overlaps, idx_returns, left_on='INDEXID', right_on='INDEXID', how='left', suffixes=('', '_idx_return'))

    # Merge the result with specific ETF returns
    final_analysis = pd.merge(etf_index_overlaps_with_returns, specific_etf_returns[['DATE', 'ETFSYMBOL', 'ETFRETURN1D']], on=['DATE'], how='left')

    # Rename and clean up columns for clarity
    final_analysis.rename(columns={
        'INDEXID': 'INDEXID_x'
    }, inplace=True)

    # Add 'InstrumentType' column for consistency
    final_analysis['InstrumentType'] = 'ETF'

    # Select relevant columns to match your desired output format
    etf_to_index_analysis = final_analysis[['DATE', 'INDEXID_x', 'ETFSymbol', 'OVERLAP', 'ETFRETURN1D', 'INDEXRETURN1D', 'InstrumentType']]
    
    return etf_to_index_analysis


#idx_id='ETFBIS|EWP'
#combined_statistics = complete_etf_to_index_returns_analysis(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)
#print(combined_statistics)

In [41]:
#Calculate combined index and etf

In [42]:
import numpy as np
import pandas as pd

def calculate_combined_statistics(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns):
    """
    Calculate statistics for both index-to-index and index-to-ETF overlaps,
    and merge back into a unified dataset.

    Parameters:
    - idx_id: The ID of the index to analyze.
    - idx_cons: DataFrame containing index constituents information.
    - etf_cons: DataFrame containing ETF constituents information.
    - idx_catalog: DataFrame with index and ETF catalog information.
    - etf_returns: DataFrame containing ETF returns information.
    - idx_returns: DataFrame containing index returns information.

    Returns:
    - DataFrame with merged statistics for both index-to-index and index-to-ETF comparisons.
    """
    # Generate index-to-index overlaps and returns analysis
    idx_overlaps = analyze_index_overlap(idx_id, idx_cons, idx_catalog)
    etf_overlaps= analyze_index_etf_overlap(idx_id, idx_cons, etf_cons, idx_catalog)
    idx_analysis = complete_analysis(idx_id, idx_cons, idx_catalog, idx_returns)
    
    etf_analysis = complete_etf_analysis(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)

#combine etf analysis
    

    etf_analysis.rename(columns={
        'ETFSymbol': 'MatchedIndex',
        'ETFRETURN1D': 'MatchedIndexReturn',
        'INDEXRETURN1D': 'OriginalIndexReturn',
        'DATE':'Date_return_a',
        'Date_return':'Date_return_b',
        'INDEXID_x':'OriginalIndex'
    }, inplace=True)


    # Ensure DATE columns are standardized and correctly formatted
    etf_analysis['Date_return_a'] = pd.to_datetime(etf_analysis['Date_return_a'])
    

    # Combine the index-to-index and index-to-ETF analysis data
    combined_analysis = pd.concat([idx_analysis, etf_analysis], ignore_index=True)

    # Calculate statistics (mean, std deviation, covariance, correlation) on the combined dataset
    # [Your existing statistical calculations here, adapted for the combined_analysis DataFrame]

    # Ensure to adjust the column names and calculations to align with the combined structure
    # For example, you might need to handle differently named columns for returns or identifiers
# Calculate Means
    means = combined_analysis.groupby(['OriginalIndex', 'MatchedIndex']).agg(
        Avg_Original=('OriginalIndexReturn', 'mean'),
        Avg_Matched=('MatchedIndexReturn', 'mean')
    ).reset_index()



    # Join back for deviation calculations
    combined = pd.merge(combined_analysis, means, on=['OriginalIndex', 'MatchedIndex'])

    
    # Calculate deviations
    combined['Deviation_Original'] = combined['OriginalIndexReturn'] - combined['Avg_Original']
    combined['Deviation_Matched'] = combined['MatchedIndexReturn'] - combined['Avg_Matched']

    # Standard Deviations within each group
    std_devs = combined.groupby(['OriginalIndex', 'MatchedIndex']).agg(
        StdDev_Original=('Deviation_Original', lambda x: np.sqrt(np.sum(x**2) / (x.count()-1))),
        StdDev_Matched=('Deviation_Matched', lambda x: np.sqrt(np.sum(x**2) / (x.count()-1)))
    ).reset_index()

    
    # Correct Covariance Calculation within each group
    def calculate_group_covariance(group):
        denominator = (len(group) - 1)
        if denominator > 0:
            return np.sum(group['Deviation_Original'] * group['Deviation_Matched']) / denominator
        else:
            return np.nan  # Or appropriate default value, like 0

    covariance = combined.groupby(['OriginalIndex', 'MatchedIndex']).apply(calculate_group_covariance).reset_index(name='Cov_AB')

    # Merge Covariance and Standard Deviations for Correlation Calculation
    correlation = pd.merge(covariance, std_devs, on=['OriginalIndex', 'MatchedIndex'])

    # Calculate Correlation Coefficient for each group
    correlation['CorrelationCoefficient'] = correlation['Cov_AB'] / (correlation['StdDev_Original'] * correlation['StdDev_Matched'])

    
    # Index specific - remove return types
    idx_statistics = pd.merge(idx_overlaps, correlation, 
                           left_on=['INDEXID_x', 'INDEXID'], 
                           right_on=['OriginalIndex', 'MatchedIndex'], 
                           how='left')

    
    def extract_before_second_pipe(index_id):
        parts = index_id.split('|')
        return '|'.join(parts[:2])

    idx_statistics['OriginalIndex_BeforeSecondPipe'] = idx_statistics['OriginalIndex'].apply(extract_before_second_pipe)
    idx_statistics['MatchedIndex_BeforeSecondPipe'] = idx_statistics['MatchedIndex'].apply(extract_before_second_pipe)

    idx_statistics = idx_statistics[idx_statistics['OriginalIndex_BeforeSecondPipe'] != idx_statistics['MatchedIndex_BeforeSecondPipe']]

    #calculate etf statistics

    etf_statistics = pd.merge(etf_overlaps, correlation, 
                           left_on='ETFSymbol', right_on='MatchedIndex', 
                           how='left')
    
    etf_statistics = pd.merge(etf_statistics, etf_catalog, 
                           left_on='ETFSymbol', right_on='ETFSYMBOL', 
                           how='left')



    # Select relevant columns
    idx_statistics = idx_statistics[['OriginalIndex', 'MatchedIndex', 'DESCRIPTION', 'OVERLAP', 'CorrelationCoefficient']]

    etf_statistics = pd.merge(
        etf_statistics,
        idx_catalog[['INDEXID', 'DESCRIPTION']].rename(columns={'DESCRIPTION': 'OriginalIndexDescription', 'INDEXID': 'OriginalIndexID'}),
        left_on='OriginalIndex',
        right_on='OriginalIndexID',
        how='left'
    )

    etf_statistics['MatchedInstrumentType']='ETF'
    
    idx_statistics = pd.merge(
        idx_statistics,
        idx_catalog[['INDEXID', 'DESCRIPTION']].rename(columns={'DESCRIPTION': 'OriginalIndexDescription', 'INDEXID': 'OriginalIndexID'}),
        left_on='OriginalIndex',
        right_on='OriginalIndexID',
        how='left'
    )

    
    idx_statistics['MatchedInstrumentType']='Index'
    
    combined_statistics = pd.concat([etf_statistics, idx_statistics], ignore_index=True)

    combined_statistics.rename(columns={
        'DESCRIPTION': 'MatchedDescription',
        'OVERLAP': 'OverlapPercentage',
        'MatchedIndex': 'MatchedID',
        'CorrelationCoefficient': 'PearsonCoefficient'
    }, inplace=True)


#convert overlap to percentage

    combined_statistics['OverlapPercentage'] = combined_statistics['OverlapPercentage']*100
    
    combined_statistics = combined_statistics[[
        'MatchedInstrumentType',
        'OriginalIndexDescription',
        'MatchedDescription',
        'OverlapPercentage',
        'PearsonCoefficient',
        'OriginalIndexID',
        'MatchedID'
    ]]
    
    return combined_statistics
#ETFBIS|EWP 'STXX|ESTX|PI'


In [43]:
import numpy as np
import pandas as pd

def calculate_etf_statistics(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns):
    """
    Calculate statistics for both index-to-index and index-to-ETF overlaps,
    and merge back into a unified dataset.

    Parameters:
    - idx_id: The ID of the index to analyze.
    - idx_cons: DataFrame containing index constituents information.
    - etf_cons: DataFrame containing ETF constituents information.
    - idx_catalog: DataFrame with index and ETF catalog information.
    - etf_returns: DataFrame containing ETF returns information.
    - idx_returns: DataFrame containing index returns information.

    Returns:
    - DataFrame with merged statistics for both index-to-index and index-to-ETF comparisons.
    """
    # Generate index-to-index overlaps and returns analysis
    etf_overlaps= analyze_etf_index_overlap(idx_id, idx_cons, etf_cons, idx_catalog)
    etf_to_index_analysis = complete_etf_to_index_returns_analysis(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)

#combine etf analysis
    
    
    etf_to_index_analysis.rename(columns={
        'ETFRETURN1D': 'MatchedIndexReturn',
        'INDEXRETURN1D': 'OriginalIndexReturn',
        'DATE':'Date_return_a',
        'Date_return':'Date_return_b',
        'INDEXID_x':'OriginalIndex',
        'ETFSymbol':'MatchedIndex'
    }, inplace=True)

    # Ensure DATE columns are standardized and correctly formatted
    etf_to_index_analysis['Date_return_a'] = pd.to_datetime(etf_to_index_analysis['Date_return_a'])
    
    # Calculate statistics (mean, std deviation, covariance, correlation) on the combined dataset
    # [Your existing statistical calculations here, adapted for the combined_analysis DataFrame]

    # Ensure to adjust the column names and calculations to align with the combined structure
    # For example, you might need to handle differently named columns for returns or identifiers
# Calculate Means
    means = etf_to_index_analysis.groupby(['MatchedIndex', 'OriginalIndex']).agg(
        Avg_Original=('MatchedIndexReturn', 'mean'),
        Avg_Matched=('OriginalIndexReturn', 'mean')
    ).reset_index()

    

    # Join back for deviation calculations
    combined = pd.merge(etf_to_index_analysis, means, on=['OriginalIndex', 'MatchedIndex'])

    
    # Calculate deviations
    combined['Deviation_Original'] = combined['OriginalIndexReturn'] - combined['Avg_Original']
    combined['Deviation_Matched'] = combined['MatchedIndexReturn'] - combined['Avg_Matched']

    # Standard Deviations within each group
    std_devs = combined.groupby(['OriginalIndex', 'MatchedIndex']).agg(
        StdDev_Original=('Deviation_Original', lambda x: np.sqrt(np.sum(x**2) / (x.count()-1))),
        StdDev_Matched=('Deviation_Matched', lambda x: np.sqrt(np.sum(x**2) / (x.count()-1)))
    ).reset_index()

    
    # Correct Covariance Calculation within each group
    def calculate_group_covariance(group):
        denominator = (len(group) - 1)
        if denominator > 0:
            return np.sum(group['Deviation_Original'] * group['Deviation_Matched']) / denominator
        else:
            return np.nan  # Or appropriate default value, like 0

    covariance = combined.groupby(['OriginalIndex', 'MatchedIndex']).apply(calculate_group_covariance).reset_index(name='Cov_AB')

    # Merge Covariance and Standard Deviations for Correlation Calculation
    correlation = pd.merge(covariance, std_devs, on=['OriginalIndex', 'MatchedIndex'])

    # Calculate Correlation Coefficient for each group
    correlation['CorrelationCoefficient'] = correlation['Cov_AB'] / (correlation['StdDev_Original'] * correlation['StdDev_Matched'])
    
     #Index specific - remove return types
    
    etf_statistics = pd.merge(etf_overlaps, correlation, 
                           left_on='ETFSymbol', right_on='MatchedIndex', 
                           how='left')
    

    #calculate etf statistics


    etf_statistics = pd.merge(etf_statistics, etf_catalog, 
                           left_on='ETFSymbol', right_on='ETFSYMBOL', 
                           how='left')


    etf_statistics = pd.merge(
        etf_statistics,
        idx_catalog[['INDEXID', 'DESCRIPTION']].rename(columns={'DESCRIPTION': 'OriginalIndexDescription', 'INDEXID': 'OriginalIndexID'}),
        left_on='OriginalIndex',
        right_on='OriginalIndexID',
        how='left'
    )

    etf_statistics['MatchedInstrumentType']='Index'
    

    etf_statistics.rename(columns={
        'DESCRIPTION': 'OriginalDescription',
        'OriginalIndexDescription':'MatchedDescription',
        'OVERLAP': 'OverlapPercentage',
        'MatchedIndex': 'OriginalID',
        'OriginalIndexID':'MatchedID',
        'CorrelationCoefficient': 'PearsonCoefficient'
        
    }, inplace=True)


#convert overlap to percentage

    etf_statistics['OverlapPercentage'] = etf_statistics['OverlapPercentage']*100
    
    etf_statistics = etf_statistics[[
        'MatchedInstrumentType',
        'OriginalDescription',
        'MatchedDescription',
        'OverlapPercentage',
        'PearsonCoefficient',
        'OriginalID',
        'MatchedID'
    ]]
    
    return etf_statistics
#ETFBIS|EWP 'STXX|ESTX|PI'
#idx_id='ETFBIS|EWP'
#combined_statistics = calculate_etf_statistics(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)
#print(combined_statistics)

In [44]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming the necessary DataFrames and functions are defined
filtered_idx_catalog = idx_catalog[idx_catalog['INDEXID'].isin(idx_cons['INDEXID'].unique())]
filtered_idx_catalog = filtered_idx_catalog[~filtered_idx_catalog['INDEXID'].str.contains('CRPI', case=False)]

# Checkboxes for selecting InstrumentType
checkbox_index = widgets.Checkbox(value=True, description='Index')
checkbox_etf = widgets.Checkbox(value=True, description='ETF')
checkbox_alternatives = widgets.Checkbox(value=True, description='Alternatives')

# Dropdown for selecting InstrumentType
dropdown_type = widgets.Dropdown(
    options=['Index', 'ETF'],
    description='Type:'
)

# Text input for filtering IndexID or ETF Symbol
text_filter = widgets.Text(
    value='',
    placeholder='Type to filter',
    description='Filter:',
    disabled=False
)

# Dropdown for displaying filtered IndexID or ETF Symbol; initially empty
dropdown_index = widgets.Dropdown(
    options=[],
    description='INDEXID:'
)

# Initial setup for dropdown options based on the default selection
display_to_original = {}

def update_dropdown_options(*args):
    global display_to_original
    display_to_original.clear()
    filter_text = text_filter.value.upper()
    options = []
    unique_display_texts = set()  # Set to keep track of unique display texts

    if dropdown_type.value == 'Index':
        for _, row in filtered_idx_catalog.iterrows():
            symbol = row['INDEXID'].split('|')[1] if '|' in row['INDEXID'] else row['INDEXID']
            display_text = f"{row['DESCRIPTION']} ({symbol})"
            if (filter_text in row['DESCRIPTION'].upper() or filter_text in row['INDEXID'].upper()) and (display_text not in unique_display_texts):
                options.append((display_text, row['INDEXID']))  # Store with original INDEXID for later retrieval
                display_to_original[display_text] = row['INDEXID']
                unique_display_texts.add(display_text)  # Mark this display text as seen
    elif dropdown_type.value == 'ETF':
        for _, row in etf_catalog.iterrows():
            symbol = row['ETFSYMBOL'].split('|')[1] if '|' in row['ETFSYMBOL'] else row['ETFSYMBOL']
            display_text = f"{row['DESCRIPTION']} ({symbol})"
            if (filter_text in row['DESCRIPTION'].upper() or filter_text in row['ETFSYMBOL'].upper()) and (display_text not in unique_display_texts):
                options.append((display_text, row['ETFSYMBOL']))  # Store with original ETFSYMBOL for later retrieval
                display_to_original[display_text] = row['ETFSYMBOL']
                unique_display_texts.add(display_text)  # Mark this display text as seen

    # Sort options by DESCRIPTION part (the text before ' (')
    sorted_options = sorted(options, key=lambda x: x[0].split(' (')[0])

    # Update dropdown options; the value displayed to the user is the first element of each tuple
    dropdown_index.options = [option[0] for option in sorted_options]


def on_filter_change(change):
    with output:
        output.clear_output()
        # Retrieve the original identifier from the selected display text
        selected_display_text = dropdown_index.value
        idx_id = display_to_original.get(selected_display_text, "")
        # Determine which instrument types are selected
        selected_types = []
        if checkbox_index.value:
            selected_types.append('Index')
        if checkbox_etf.value:
            selected_types.append('ETF')
        # Call the combined_statistics function with the selected idx_id
        if dropdown_type.value == 'Index':
            combined_statistics = calculate_combined_statistics(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)
        elif dropdown_type.value == 'ETF':
            combined_statistics = calculate_etf_statistics(idx_id, idx_cons, etf_cons, idx_catalog, etf_returns, idx_returns)
        # Filter combined_statistics by selected instrument types
        if selected_types:
            filtered_statistics = combined_statistics[combined_statistics['MatchedInstrumentType'].isin(selected_types)]
        else:
            filtered_statistics = pd.DataFrame()  # Empty DataFrame if no types are selected
        
        # Calculate the index of the max 'PearsonCoefficient' for each 'MatchedDescription'
        max_idx = filtered_statistics.groupby('MatchedDescription')['PearsonCoefficient'].idxmax()
        
        # Drop NaN values from max_idx to avoid KeyError when using .loc[]
        max_idx = max_idx.dropna()
        
        # Use the filtered max_idx to index into filtered_statistics
        filtered_statistics = filtered_statistics.loc[max_idx]
        if not filtered_statistics.empty:
            # Sort by custom criterion: (OverlapPercentage/100) + PearsonCoefficient
            filtered_statistics = (
                filtered_statistics.assign(
                CustomSortCriterion=lambda x: (x['OverlapPercentage']/100) + x['PearsonCoefficient']
            )
            .sort_values(by='CustomSortCriterion', ascending=False)
)

            top_filtered_statistics = filtered_statistics.head(20)

# Optionally, if you want to drop the CustomSortCriterion column from the top 20 results
            top_filtered_statistics = top_filtered_statistics.drop(columns=['CustomSortCriterion'])
            display(top_filtered_statistics)
        
# Call update_dropdown_options when the type is changed or text is entered for filtering
dropdown_type.observe(update_dropdown_options, 'value')
text_filter.observe(update_dropdown_options, 'value')
dropdown_index.observe(on_filter_change, names='value')
checkbox_index.observe(on_filter_change, names='value')
checkbox_etf.observe(on_filter_change, names='value')
checkbox_alternatives.observe(on_filter_change, names='value')
dropdown_index.observe(on_filter_change, names='value')
checkbox_index.observe(on_filter_change, names='value')
checkbox_etf.observe(on_filter_change, names='value')
checkbox_alternatives.observe(on_filter_change, names='value')

# Initial call to populate dropdown options
update_dropdown_options()

# Placeholder to display the DataFrame or any other output
output = widgets.Output()

# Your existing logic for on_value_change and on_filter_change here

# Display the widgets and output placeholder
display(dropdown_type, text_filter, dropdown_index, checkbox_index, checkbox_etf, checkbox_alternatives, output)


Dropdown(description='Type:', options=('Index', 'ETF'), value='Index')

Text(value='', description='Filter:', placeholder='Type to filter')

Dropdown(description='INDEXID:', options=('COMMUNICATION SERVICES SELECT SECTOR INDEX (5S25.US.SC00L.50)', 'CO…

Checkbox(value=True, description='Index')

Checkbox(value=True, description='ETF')

Checkbox(value=True, description='Alternatives')

Output()